In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import os

import datetime

import pandas as pd

from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [8]:
dfMissingPM25 = pd.read_csv('../MissingPM25Date-Province/MissingPM25ForEachDateForEachProvince.csv')
dfMissingPM25

,Unnamed: 0,date,provinceNo
0,0,20200722,40.0
1,0,20200722,44.0
2,0,20200722,31.0
3,0,20200722,8.0
4,0,20200722,45.0
...,...,...,...
689,0,20220108,11.0
690,0,20220115,70.0
691,0,20220115,31.0
692,0,20220122,70.0


In [9]:
dfMissingPM25 = dfMissingPM25.drop(columns=dfMissingPM25.columns[0], axis=1)
dfMissingPM25

,date,provinceNo
0,20200722,40.0
1,20200722,44.0
2,20200722,31.0
3,20200722,8.0
4,20200722,45.0
...,...,...
689,20220108,11.0
690,20220115,70.0
691,20220115,31.0
692,20220122,70.0


In [10]:
dfMissingPM25['provinceNo'] = dfMissingPM25['provinceNo'].astype(int)
dfMissingPM25

,date,provinceNo
0,20200722,40
1,20200722,44
2,20200722,31
3,20200722,8
4,20200722,45
...,...,...
689,20220108,11
690,20220115,70
691,20220115,31
692,20220122,70


In [11]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()
provinceDict = {74:["yala","6.506;101.341;9"], 71:["satun","6.680;100.403;9"], 72:["songkhla","7.125;100.658;9"], 63:["trang","7.493;99.926;9"], \
65-1:["nakhon-si-thammarat","8.227;99.789;9"],70-1:["wichit","8.038;98.722;9"],74-1:["surat-thani","9.115;99.007;9"],71-1:["ranong","9.863;98.867;9"], \
57-1:["kanchanaburi","13.923;100.349;9"] , 44-1:["samut-sakhon","13.931;100.267;9"], 76-1:["bangkok","13.931;100.267;9"], 42-1:["samut-prakan","13.931;100.267;9"], \
"mueang-nonthaburi":[35-1,"13.931;100.267;9"], 36-1:["pathum-thani","13.931;100.267;9"], 37-1: ["phra-nakhon-si-ayutthaya","13.931;100.267;9"],\
48-1:["saraburi","14.475;100.437;9"] , \
41-1:["lop-buri","14.475;100.437;9"] , 45-1:["sing-buri","14.475;100.437;9"] , 54-1:["prachin-buri","14.475;100.437;9"] , 32-1:["nakhon-nayok","14.475;100.437;9"] , \
1-1:["chiang-rai", "19.22;99.99;8"], 6-1:["mae-hong-son", "19.02;97.87;8"], 4-1:["phayao", "19.10;100.27;8"], 3-1:["nan", "18.79;100.80;8"], \
2-1:["chiang-mai", "18.93;99.06;8"], 8-1:["lamphun", "18.50;99.07;8"], 7-1:["lampang", "18.35;99.42;8"], 5-1:["phrae", "18.17;100.26;8"], 9-1:["uttaradit", "17.55;100.10;8"], \
46-1:["sukhothai", "16.96;99.97;8"], 58-1:["tak", "16.83;99.18;8"], 39-1:["phitsanulok", "16.83;100.20;8"], 30-1:["kamphaeng-phet", "16.35;99.30;8"], 38-1:["phichit", "16.37;100.45;8"], \
40-1:["phetchabun", "16.50;101.39;8"], 34-1:["nakhon-sawan", "15.73;100.26;8"], 50-1:["uthai-thani", "15.35;100.12;8"], 25-1:["nong-khai", "17.91;102.82;8"], \
21-1:["loei", "17.47;101.90;8"], 27-1:["udon-thani", "17.50;102.79;8"], 13-1:["nakhon-phanom", "17.41;104.73;8"], \
22-1:["sakon-nakhon", "17.17;104.19;8"], 11-1:["khon-kaen", "16.34;102.69;8"], 10-1:["kalasin", "16.50;103.73;8"], \
20-1:["roi-et", "16.10;103.69;8"],  12-1:["chaiyaphum", "15.87;102.05;8"], \
28-1:["ubon-ratchathani", "14.98;105.17;8"], 16-1:["buriram", "14.84;102.88;8"], \
14-1:["nakhon-ratchasima","14.85;102.04;8"], 77-1:["chon-buri", "13.25;101.13;8"], 55-1:["rayong", "12.67;101.21;8"], 52:["trat", "12.25;102.55;8"]}



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been 

In [12]:
def findStuff(date,provinceNo):
    # firstTime=True
    url = f'https://www.ventusky.com/?p={provinceDict[provinceNo][1]}&l=pm25&t={date}/0600'
    driver.get(url=url) 
    #wait for web to load
    # if(firstTime):
    # time.sleep(2)
    #     firstTime=False
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"a[href*={provinceDict[provinceNo][0]}]")))
        print ("Page is ready!")
    except TimeoutException:
        print ("Loading took too much time!")
    ozoneValue = driver.find_elements_by_css_selector(f'a[href={provinceDict[provinceNo][0]}]')[0]
    driver.execute_script("arguments[0].click();", ozoneValue)
    try:
        # WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, f'a[href={key}]')))
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, f'a[class="qo am zh"][href="{provinceDict[provinceNo][0]}"]')))
        # WebDriverWait(driver,10).until(EC.presence_of_element_located(driver.find_elements_by_css_selector(f'a[class="qo am zh"][href="{key}"]')[0]))
    except:
        print("not found href 1")
        # try:
        #     WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, f'a[class="qo am i zh"][href="{key}"]')))
        # except:
        #     print("not found href 2")
    time.sleep(1)
    ozoneValue = driver.find_elements_by_css_selector(f'a[href={provinceDict[provinceNo][0]}]')
    realOzoneValue=ozoneValue[-1]
    for e in ozoneValue:
        testValue = e.get_attribute("data-value")
        if(testValue!=None):
            realOzoneValue=e
            break
        else:
            print(testValue)
    # ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')[0]
    #print("------------------")
    # print(ozoneValue.text)
    # print(ozoneValue.get_attribute('style'))
    scrapedValue = [date,provinceNo,realOzoneValue.get_attribute('data-value').split(" ")[0]]
    print(realOzoneValue.get_attribute('data-value').split(" ")[0])
    return scrapedValue
    

In [13]:
missingPM25 = dfMissingPM25.values.tolist()
missingPM25

[[20200722, 40],
 [20200722, 44],
 [20200722, 31],
 [20200722, 8],
 [20200722, 45],
 [20200722, 38],
 [20200722, 29],
 [20200722, 37],
 [20200722, 39],
 [20200722, 26],
 [20200722, 12],
 [20200722, 21],
 [20200722, 9],
 [20200722, 19],
 [20200722, 11],
 [20200722, 15],
 [20200801, 40],
 [20200801, 44],
 [20200801, 31],
 [20200801, 8],
 [20200801, 45],
 [20200801, 38],
 [20200801, 29],
 [20200801, 37],
 [20200801, 39],
 [20200801, 26],
 [20200801, 12],
 [20200801, 21],
 [20200801, 9],
 [20200801, 19],
 [20200801, 11],
 [20200801, 15],
 [20200808, 40],
 [20200808, 44],
 [20200808, 31],
 [20200808, 8],
 [20200808, 45],
 [20200808, 38],
 [20200808, 29],
 [20200808, 37],
 [20200808, 39],
 [20200808, 26],
 [20200808, 12],
 [20200808, 21],
 [20200808, 9],
 [20200808, 19],
 [20200808, 11],
 [20200808, 15],
 [20200815, 63],
 [20200815, 40],
 [20200815, 44],
 [20200815, 31],
 [20200815, 8],
 [20200815, 45],
 [20200815, 38],
 [20200815, 29],
 [20200815, 37],
 [20200815, 39],
 [20200815, 26],
 [20

In [15]:
tempPM25 = []
for e in missingPM25:
    tempPM25.append(findStuff(e[0],e[1]))
dfPM25Result = pd.DataFrame(
    data=tempPM25,
    columns=['date','provinceNo','PM25']
)
dfPM25Result

Page is ready!


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17580\1820575178.py:14: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={provinceDict[provinceNo][0]}]')[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17580\1820575178.py:27: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={provinceDict[provinceNo][0]}]')


37
Page is ready!
None
52
Page is ready!
None
33
Page is ready!
38
Page is ready!
31
Page is ready!
31
Page is ready!
38
Page is ready!
30
Page is ready!
13
Page is ready!
18
Page is ready!
23
Page is ready!
20
Page is ready!
25
Page is ready!
21
Page is ready!
10
Page is ready!
13
Page is ready!
6
Page is ready!
None
4
Page is ready!
None
12
Page is ready!
7
Page is ready!
3
Page is ready!
7
Page is ready!
36
Page is ready!
12
Page is ready!
3
Page is ready!
11
Page is ready!
13
Page is ready!
16
Page is ready!
24
Page is ready!
20
Page is ready!
11
Page is ready!
12
Page is ready!
18
Page is ready!
None
10
Page is ready!
None
45
Page is ready!
25
Page is ready!
26
Page is ready!
19
Page is ready!
32
Page is ready!
14
Page is ready!
15
Page is ready!
32
Page is ready!
22
Page is ready!
30
Page is ready!
30
Page is ready!
24
Page is ready!
16
Page is ready!
25
Page is ready!
42
Page is ready!
42
Page is ready!
None
24
Page is ready!
None
20
Page is ready!
18
Page is ready!
16
Page is r

,date,provinceNo,PM25
0,20200722,40,37
1,20200722,44,52
2,20200722,31,33
3,20200722,8,38
4,20200722,45,31
...,...,...,...
689,20220108,11,22
690,20220115,70,15
691,20220115,31,24
692,20220122,70,7


In [16]:
dfPM25Result.to_csv('../ScrapedData/PM25OfMissingDateAndProvince.csv')